In [19]:
import os
from dotenv import load_dotenv

import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build

# ── CONFIG ─────────────────────────────────────────────────────────────────────
load_dotenv()
KEY_PATH    = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")  # e.g. "path/to/sa.json"
PROJECT     = os.getenv("GCP_PROJECT")                    # your GCP project ID
SERVICE_ID  = os.getenv("SERVICE_ID", "6F81-5844-456A")    # Compute Engine billing service
BILLING_API = f"services/{SERVICE_ID}"

In [20]:
creds   = service_account.Credentials.from_service_account_file(KEY_PATH)
compute = build("compute", "v1", credentials=creds)
billing = build("cloudbilling", "v1", credentials=creds)

In [21]:
def get_category_definitions():
    return {
        'General Purpose':     {'c4a','c4','c4d','c3','c3d','n4','n2','n2d','n1','g1','t2a','t2d','e2','f1'},
        'Storage Optimized':   {'z3'},
        'Compute Optimized':   {'h3','c2','c2d'},
        'Memory Optimized':    {'x4','m4','m3','m2','m1'},
        'Accelerator Optimized': {'a4','a3','a2','g2'},
        'Tensor Processing Unit': {'ct3','ct3p','ct4p','ct5l','ct5p','ct6e'},
    }

In [22]:
_category_map = {
    fam: cat
    for cat, fams in get_category_definitions().items()
    for fam in fams
}

In [23]:
def collect_all_machine_types():
    rows = []
    req = compute.machineTypes().aggregatedList(project=PROJECT)
    while req:
        resp = req.execute()
        for zone_uri, zone_block in resp.get("items", {}).items():
            zone   = zone_uri.split("/")[-1]
            region = zone.rsplit("-", 1)[0]
            for m in zone_block.get("machineTypes", []):
                family   = m["name"].split("-", 1)[0]
                category = _category_map.get(family, "Other")
                rows.append({
                    "Region":         region,
                    "MachineType":    m["name"],
                    "Machine Family": category,
                    "vCPUs":          m["guestCpus"],
                    "MemoryMiB":      m["memoryMb"],
                    "MemoryGiB":      round(m["memoryMb"] / 1024, 2),
                    "Accelerators":   m.get("accelerators"),  # list of dicts or None
                })
        req = compute.machineTypes().aggregatedList_next(previous_request=req, previous_response=resp)
    return pd.DataFrame(rows)

In [32]:
def fetch_compute_skus():
    skus = []
    req = billing.services().skus().list(
        parent=BILLING_API,
        pageSize=5000,
        currencyCode="USD",
    )
    while req:
        resp = req.execute()
        skus.extend(resp.get("skus", []))
        req = billing.services().skus().list_next(previous_request=req, previous_response=resp)

    data = []
    for sku in skus:
        cat = sku.get("category", {})
        # only on-demand Compute & GPU SKUs
        if cat.get("usageType") != "OnDemand" or cat.get("resourceFamily") not in ("Compute", "GPU"):
            continue

        expr = sku["pricingInfo"][0]["pricingExpression"]
        unit_price = expr["tieredRates"][0]["unitPrice"]
        # CAST to numbers
        units = float(unit_price.get("units", "0"))
        nanos = float(unit_price.get("nanos", 0))
        price = units + nanos / 1e9

        base = {
            "ResourceGroup": cat.get("resourceGroup"),  # e.g. "CPU", "RAM", "NVIDIA_TESLA_V100"
            "UsageUnit":     expr.get("usageUnit"),     # e.g. "h"
            "PricePerUnit":  price,                     # USD per hour
        }
        for region in sku.get("serviceRegions", []):
            row = base.copy()
            row["Region"] = region
            data.append(row)

    return pd.DataFrame(data)


In [33]:
def merge_prices_and_compute(df_machines, df_skus):
    # Pivot CPU & RAM prices
    cpu = df_skus[df_skus.ResourceGroup == "CPU"][["Region","PricePerUnit"]].rename(columns={"PricePerUnit":"CpuPrice"})
    ram = df_skus[df_skus.ResourceGroup == "RAM"][["Region","PricePerUnit"]].rename(columns={"PricePerUnit":"RamPrice"})
    # All non-CPU/RAM SKUs are GPUs or TPUs
    gpu = df_skus[~df_skus.ResourceGroup.isin({"CPU","RAM"})][["Region","ResourceGroup","PricePerUnit"]]

    df = df_machines.merge(cpu, on="Region", how="left")
    df = df.merge(ram, on="Region", how="left")

    # Compute GPU/TU cost
    def gpu_cost(acc_list, region):
        if not acc_list:
            return 0.0
        total = 0.0
        for a in acc_list:
            type_key = a["acceleratorType"].split("/")[-1].upper().replace("-", "_")
            cnt      = a["acceleratorCount"]
            match    = gpu[
                (gpu.Region == region) &
                (gpu.ResourceGroup == type_key)
            ]["PricePerUnit"]
            if not match.empty:
                total += float(match.iloc[0]) * cnt
        return total

    df["GpuPrice"]      = df.apply(lambda r: gpu_cost(r["Accelerators"], r["Region"]), axis=1)
    df["CostPerHour"]   = (
        df["vCPUs"] * df["CpuPrice"] +
        df["MemoryGiB"] * df["RamPrice"] +
        df["GpuPrice"]
    ).round(6)

    return df

In [34]:
machines_df = collect_all_machine_types()

In [36]:
print(f"Fetched {len(machines_df)} machine‐type entries.")

Fetched 24184 machine‐type entries.


In [38]:
skus_df = fetch_compute_skus()
print(f"Got {len(skus_df)} on-demand SKU rows")

Got 6940 on-demand SKU rows


In [39]:
def merge_prices_and_compute(df_machines, df_skus):
    # pivot CPU & RAM
    cpu = df_skus[df_skus.ResourceGroup=="CPU"][["Region","PricePerUnit"]].rename(columns={"PricePerUnit":"CpuPrice"})
    ram = df_skus[df_skus.ResourceGroup=="RAM"][["Region","PricePerUnit"]].rename(columns={"PricePerUnit":"RamPrice"})
    gpu = df_skus[~df_skus.ResourceGroup.isin({"CPU","RAM"})][["Region","ResourceGroup","PricePerUnit"]]

    df = df_machines.merge(cpu, on="Region", how="left")\
                    .merge(ram, on="Region", how="left")

    def gpu_cost(acc_list, region):
        if not acc_list:
            return 0.0
        total = 0.0
        for a in acc_list:
            # normalize e.g. "nvidia-tesla-v100" -> "NVIDIA_TESLA_V100"
            key = a["acceleratorType"].split("/")[-1].upper().replace("-", "_")
            cnt = a["acceleratorCount"]
            match = gpu[(gpu.Region==region)&(gpu.ResourceGroup==key)]["PricePerUnit"]
            if not match.empty:
                total += float(match.iloc[0])*cnt
        return total

    df["GpuPrice"]    = df.apply(lambda r: gpu_cost(r["Accelerators"], r["Region"]), axis=1)
    df["CostPerHour"] = (df["vCPUs"]*df["CpuPrice"] + df["MemoryGiB"]*df["RamPrice"] + df["GpuPrice"]).round(6)
    return df

In [ ]:
result_df = merge_prices_and_compute(machines_df, skus_df)